In [ ]:
from pyspark.sql import SparkSession, dataframe
from pyspark.sql.functions import when, col, sum, count, isnan, round
from pyspark.sql.functions import regexp_replace, concat_ws, sha2, rtrim, substring
from pyspark.sql.functions import unix_timestamp, from_unixtime, to_date
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType, StringType
from pyspark.sql import HiveContext
from pyspark.sql.functions import year, month, dayofmonth, quarter


import os
import re

from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import when

spark = SparkSession.builder.master("local[*]")\
    .enableHiveSupport()\
    .getOrCreate()

In [ ]:
def salvar_df(df, file):
    output = "/input/projeto_hive/gold/" + file
    erase = "hdfs dfs -rm " + output + "/*"
    rename = "hdfs dfs -get /datalake/gold/"+file+"/part-* /input/projeto_hive/gold/"+file+".csv"
    print(rename)
    
    
    df.coalesce(1).write\
        .format("csv")\
        .option("header", True)\
        .option("delimiter", ";")\
        .mode("overwrite")\
        .save("/datalake/gold/"+file+"/")

    os.system(erase)
    os.system(rename)

In [ ]:
df_clientes = spark.sql("select * from desafio_curso.clientes")
df_divisao = spark.sql("select * from desafio_curso.divisao")
df_endereco = spark.sql("select * from desafio_curso.endereco")
df_regiao = spark.sql("select * from desafio_curso.regiao")
df_vendas = spark.sql("select * from desafio_curso.vendas")

df_endereco.show(truncate=False)

In [ ]:
#Fazer um tabelão

df_stage_clientes_localidade = df_clientes.join(df_endereco, "address_number", "inner")\
                                .join(df_divisao, "division", "inner")\
                                .join(df_regiao, "region_code", "inner")\
                                .join(df_vendas, "customer_key", "inner")

df_stage_clientes_localidade.show()

In [ ]:
#Adicionar colunas Ano, Mês, Dia, Trimestre tomando como base a coluna invoice_date

df_stage_clientes_localidade = df_stage_clientes_localidade.withColumn('invoice_date', to_date(col('invoice_date'), 'dd/MM/yyyy'))

df_stage = df_stage_clientes_localidade \
    .withColumn('Ano', year('invoice_date')) \
    .withColumn('Mês', month('invoice_date')) \
    .withColumn('Dia', dayofmonth('invoice_date')) \
    .withColumn('Trimestre', quarter('invoice_date'))

df_stage.show() 
                        

In [ ]:
#Campos strings vazios deverão ser preenchidos com 'Não informado'

for column in df_stage.columns:
    if df_stage.schema[column].dataType in (DecimalType(), IntegerType()):
        df_stage = df_stage.withColumn(column, when(col(column).isNull() | (col(column) == ""), 0).otherwise(col(column)))
    else:
        df_stage = df_stage.withColumn(column, when(col(column) == "", "Não informado").otherwise(col(column)))
        
df_stage.show()


In [ ]:
df_stage.filter(col("address_number") == "10000453").show()
